In [1]:
# try the imports
import os
import torch
import math
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
sys.path.append("../")
from tqdm.notebook import tqdm
sys.path.append('../UI_Labeling')
from random import sample
from torch.autograd import Variable
from torch.nn import functional as F
import cv2
import json
from shapely.wkt import loads
from shapely.geometry import Point
import geopandas
from descarteslabs.scenes import DLTile
import geojson
import descarteslabs.workflows as wf
from ipyleaflet import GeoJSON
from scipy import signal
from scipy import ndimage
from torchvision.models import resnet18
import torch.nn as nn
import descarteslabs as dl

!{sys.executable} -m pip install opencv-contrib-python

/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:57: FutureWarning: 'pyarrow.SerializationContext' is deprecated and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  serialization_context = pa.SerializationContext()
/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:58: FutureWarning: 'pyarrow.register_default_serialization_handlers' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  pa.register_default_serialization_handlers(serialization_context)


Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
upsampled_model_name = "Upsampled_bipn"
upsampledlogdir = "model_log_dl/"+upsampled_model_name+"_50_training_steps/"

In [3]:
class UpsampledResnet(torch.nn.Module):
    def __init__(self, num_channels, image_width, device=None, pretrained=False):
        super(UpsampledResnet, self).__init__()
        self.device = device        
        self.resnet = resnet18(pretrained=pretrained)
        self.resnet.conv1 = torch.nn.Conv2d(num_channels, image_width, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.upsample = torch.nn.Upsample(scale_factor=8)
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, 256).to(device)
        self.final_fc = torch.nn.Linear(256, 1).to(device)
        
        self.resnet = self.resnet.to(device)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x=self.upsample(x)
        resnet_output = self.resnet(x)
        outputs = self.final_fc(resnet_output)
        outputs = self.sigmoid(outputs)
        return outputs
    
def load_checkpoint(model_checkpoint, model, device, optimizer=None):
    """
    Loads a pretrained checkpoint to continue training
    model_checkpoint: Path of the model_checkpoint that ends with .pth
    model: model to load to
    device: devide to load on (gpu/cpu)
    optimizer (optional): optimize to load state to
    """
    checkpoint = torch.load(model_checkpoint, map_location=device)
    print('Loaded best state dict from epoch: {}'.format(checkpoint["epoch"]))
    model.load_state_dict(checkpoint["model_state_dict"])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.to(device)
    return model

In [4]:
def returnCAM(feature_conv, weight_softmax, class_idx, kernel, size_upsample):
    # generate the class activation maps upsample to 256x256
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    labeled=[]
    nr_objects=[]
    centers=[]
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = np.transpose(cam)
        #normalize
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        
        #calculate mean and convolute using large mask
        #"""
        mean = np.mean(cam[np.nonzero(cam)])
        cam = cam-mean
        cam = np.clip(cam,0,None)
        #convolve=signal.convolve2d(cam,np.ones((9,9))/81)
        #cam=convolve
        cam = cv2.GaussianBlur(cam,(kernel,kernel),0)
        
        
        #calculate mean and convolute using medium mask
        #"""
        mean = np.mean(cam[np.nonzero(cam)])
        cam = cam-mean
        cam = np.clip(cam,0,None)
        
        cam = cv2.GaussianBlur(cam,(kernel,kernel),0)
        #"""
        
        #calculate mean and convolute using small mask
        #"""
        mean = np.mean(cam[np.nonzero(cam)])
        cam = cam-mean
        cam = np.clip(cam,0,None)
        #"""
        
        #calculate mean and send the final cam
        #"""
        mean = np.mean(cam[np.nonzero(cam)])
        cam = cam-mean

        #cam = np.clip(cam,0,None)
        #mean = np.mean(cam[np.nonzero(cam)])
        #cam = cam-mean
        #"""
        cam=cv2.resize(cam, (size_upsample,size_upsample))
        cam_img = np.clip(cam,0,None)
        cam_img = cam_img-(np.max(cam)*.75)
        cam_img=np.clip(cam,0,None)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cam_img)
        labeled1,nr_objects1=ndimage.label(output_cam)
        labeled.append(labeled1)
        nr_objects.append(nr_objects1)
        centers1=ndimage.measurements.center_of_mass(output_cam[-1].squeeze(),labeled[-1].squeeze(),range(1,nr_objects1+1))
        centers.append(centers1)
    return output_cam,labeled,nr_objects,centers

In [5]:
def returnOriginalCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam=np.transpose(cam)
        
        #normalize
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        
        cam_img = np.clip(cam,0,None)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cam_img)
        
    return output_cam

In [6]:
datasetName='Image_Datasets/all_2021_bangladesh_dl_dataset/'
df = pd.read_csv(datasetName+"metadata.csv")

In [7]:
upsamplednet = UpsampledResnet(3,64, device=device)
upsamplednet = load_checkpoint(f"{upsampledlogdir}/checkpoints/best_dl_best.pth", 
                                 upsamplednet, 
                                 device)
upsamplednet.to(device)
upsampled_conv_name='layer4'
upsamplednet.eval()

upsampled_features_blobs = []
def hook_feature(module, input, output):
    upsampled_features_blobs.append(output.data.cpu().numpy())
upsamplednet._modules.get('resnet')._modules.get(upsampled_conv_name).register_forward_hook(hook_feature)

upsampledparams = list(upsamplednet.parameters())
upsampled_weight_softmax = [np.dot(np.squeeze(upsampledparams[-2].data.numpy()),(np.squeeze(upsampledparams[-4].data.numpy())))]

Loaded best state dict from epoch: 10


In [8]:
m=wf.interactive.MapApp()
m.center=(25,89)
m.zoom=8
#for batch in tqdm(range(1,math.ceil(len(df['Geometry'])/500))):
    #short = df['Geometry'][batch*500:(batch+1)*500]
with tqdm(total=len(df['Geometry'])) as pbar:
    latlons={}
    for idx,x in enumerate(df['Geometry']):
        if df['Label'][idx]:
            image=df['Image'][idx]
            tile=dl.scenes.DLTile.from_key(df['Key'][idx])
            upsampledimg=np.load(datasetName+image)
            upsampledimg = np.transpose(upsampledimg,(2,0,1))
            upsampledimg=torch.Tensor(upsampledimg).to(device)
            upsampledlogit=upsamplednet(Variable(upsampledimg.unsqueeze(0)))
            upsampledh_x=F.softmax(upsampledlogit,dim=1).data.squeeze()
            upsampledprobs,upsampledidx=upsampledh_x.sort(0,True)
            upsampledprobs=upsampledprobs.numpy()
            upsampledidx=upsampledidx.numpy()
            upsampledCAMs,upsampledlabeled,upsamplednr_objects,upsampledcenters=returnCAM(upsampled_features_blobs[-1],[upsampled_weight_softmax[-1]*-1],[upsampledidx],9,upsampledimg.shape[1])
            upsampledoCAMs = returnOriginalCAM(upsampled_features_blobs[-1],[upsampled_weight_softmax[-1]*-1],[upsampledidx])
            temp=[]
            for centers in upsampledcenters[0]:
                coord=tile.rowcol_to_latlon(round(centers[0]/4),round(centers[1]/4))
                temp.append((coord[0][0],coord[1][0]))

                #Place point markers on the map
                """
                subtilegjs=GeoJSON(
                    data=geojson.FeatureCollection([geopandas.GeoSeries(Point((coord[1][0],coord[0][0]))).__geo_interface__]),
                    style={"color":'green',
                          "fillOpacity":0,
                          "weight":5}
                )
                m.add_layer(subtilegjs)
                #"""
            latlons[df['Key'][idx]]=temp
            upsampled_features_blobs=[]
            #Place squares on predicted tiles on map
            """
            temp=geopandas.GeoSeries(loads(x)).__geo_interface__
            subtilegjs=GeoJSON(
                data=geojson.FeatureCollection([temp]),
                style={"color":"red",
                      "fillOpacity":0,
                      "weight":2}
            )
            m.add_layer(subtilegjs)
            #"""

            #Display the tile image and the CAM representations
            """
            fig=plt.figure()
            fig.suptitle('Upsampled Model')
            fig.add_subplot(2,2,2)
            plt.imshow(upsampledoCAMs[-1])
            plt.title('Original CAM')
            fig.add_subplot(2,2,3)
            plt.imshow(upsampledCAMs[-1])
            plt.title('Filtered CAM')
            fig.add_subplot(2,2,4)
            plt.imshow(np.transpose(upsampledlabeled[-1],(1,2,0)))
            plt.title('Group CAM')
            fig.add_subplot(2,2,1)
            plt.imshow(np.transpose(upsampledimg.int(),(2,1,0)))
            plt.title('Original Img')
            #"""

        pbar.update()

    with open (r'2021Bangladesh_KilnCoords.json','w') as outfile:
        json.dump(latlons,outfile)
#"""

  0%|          | 0/1627 [00:00<?, ?it/s]

In [9]:
m


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.
